# COGS 118B - Final Project

# Automated Shift Scheduling

## Group members

- Immanuel Chai
- Sidney Ma
- Amy Liu

# Abstract 
The goal of the project is to automate and optimally create a scheduler for a dining hall with 100 student workers each with a randomly generated class schedule while improving the schedule over time through deep learning. The data used is a csv file that contains a list of 100 students alongside their availability for every half hour in the day starting from 6:00 am to 11:30 pm. Initially, the solution involves creating a scheduler that assigns students to shifts based on the basic requirements. The scheduler eventually optimizes to meet the other constraints. The final schedule is saved to an excel file where it highlights when each student has a shift and the scheduler is measured based on how well it can stick to the constraints while also keeping the basic requirements.

# Background

The workforce scheduling problem (WSP) involves determining the optimal assignment of workers to various work periods while satisfying constraints such as labor laws and employee preferences<sup>([1](https://www.researchgate.net/publication/2333359_Workforce_Scheduling))</sup>. This problem is crucial in maximizing organizational efficiency and profitability without compromising staff well-being.

WSP has been a significant theme in operations research due to its broad applications in industries reliant on human capital, such as healthcare, public transportation, and customer service<sup>([2](https://www.researchgate.net/publication/46189793_Heuristic_Algorithm_for_Workforce_Scheduling_Problems))</sup>. These industries require efficient scheduling to ensure high service levels and operational efficiency. In the context of higher education, one application of the WSP can be found in assigning shifts for student workers at the dining hall. This setting presents unique challenges, such as students’ class schedules, ensuring adequate staffing in opening and closing shift hours, and preventing consecutive closing and opening shifts.

Previous research has applied various algorithms and methodologies to address WSP. Techniques used for Constraint-Satisfaction Problems (CSPs) are well-suited for handling the complex and diverse constraints typical of scheduling problems<sup>([3](http://eil2.mie.utoronto.ca/wp-content/uploads/scheduling/papers/fox-ecai90.pdf))</sup>. Machine learning methods, particularly reinforcement learning, have also shown promise in dynamically adjusting schedules based on real-time data and learning from historical scheduling outcomes<sup>([4](https://www.sciencedirect.com/science/article/pii/S2212827122001846))</sup>.

# Problem Statement

This problem is an approximation of the needs of managers at the UCSD Sixth Market, where they have to assign shifts for student workers each week, based on their class schedules and a set of other constraints. Our goal is to create a tool that takes in a large dataset of student workers’ class schedules and outputs a realistic, fair, and efficient work shift schedule for a given facility. Specifically, the tool will be provided with 100 unique students and their weekly class schedules. It will need to organize a schedule such that at least 6 students are present in the facility at all open times (6am - 12 am), each shift is 4.5 hours long, each student works 3 or 4 shifts per week, and does not work more than 1 shift per day. 

As mentioned before, this problem could be appropriately modeled as a CSP. The tool’s performance will be based on how well it abides by the above constraints, measured by its reward scores. We will determine the exact preferences in the process. We also plan to replicate its performance with a wide variety of student schedules to ensure that it is generalizable to new students.

# Data

In order to train and test our tool, we will need student class schedule data to work with. While it could be possible to collect real student schedule data, this data might not be readily available to us, and it might lead to a considerable privacy violation. To avoid this, we plan to create a synthetic dataset which will model what typical UCSD student class schedules look like. Each observation of this dataset represents a student and their weekly class schedule. This includes their ID, name, and hour-by-hour availability from Sunday through Saturday, 6:00am to 11:30pm. A rough example model of this dataset can be found here: https://github.com/iachai/COGS188_schedulers/blob/main/example.csv.

While it could also be possible to collect data on real shift schedules to get a sense of what a “good schedule” looks like, this involves the same problem as before, with access and privacy issues. Most likely, we will not need this data anyway, as our solution does not involve modeling after real schedules.


In [5]:
import pandas as pd
file_path = 'example_students.csv'
students_df = pd.read_csv(file_path)
students_df.head()

,id,name,is_lead,sun0600am,sun0630am,sun0700am,sun0730am,sun0800am,sun0830am,sun0900am,...,sat0700pm,sat0730pm,sat0800pm,sat0830pm,sat0900pm,sat0930pm,sat1000pm,sat1030pm,sat1100pm,sat1130pm
0,7064287529,Amanda Washington,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,5365860717,Joshua Meyer,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,6141344355,Randy Gill,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,6332792674,Tyler Peterson,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,1790210547,Austin Glover,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


Additionally, since shifts only happen on selected times in this case, we narrowed the dataset down to a 4.5-hour resolution, which makes it easier to see availability shift-by-shift.

In [6]:
import pandas as pd

file_path = 'example_students.csv'
students_df = pd.read_csv(file_path)
days = ['sun', 'mon', 'tue', 'wed', 'thu', 'fri', 'sat']
shifts = {
    '0600am-1030am': ['0600am', '0630am', '0700am', '0730am', '0800am', '0830am', '0900am', '0930am', '1000am', '1030am'],
    '1030am-0300pm': ['1030am', '1100am', '1130am', '1200pm', '1230pm', '0100pm', '0130pm', '0200pm', '0230pm', '0300pm'],
    '0300pm-0730pm': ['0300pm', '0330pm', '0400pm', '0430pm', '0500pm', '0530pm', '0600pm', '0630pm', '0700pm', '0730pm'],
    '0730pm-1130pm': ['0730pm', '0800pm', '0830pm', '0900pm', '0930pm', '1000pm', '1030pm', '1100pm', '1130pm']
}

new_schedule_df = pd.DataFrame(columns=['id', 'name'] + [f"{day}-{shift}" for day in days for shift in shifts.keys()])
new_schedule_df['id'] = students_df['id']
new_schedule_df['name'] = students_df['name']

def check_shift_availability(student_id, day, shift):
    for hour in shifts[shift]:
        if not students_df.at[student_id, f"{day}{hour}"]:
            return False
    return True

for student_id in range(len(students_df)):
    for day in days:
        for shift in shifts:
            new_schedule_df.at[student_id, f"{day}-{shift}"] = check_shift_availability(student_id, day, shift)

            
output_file_path = 'availability_schedule.xlsx'
new_schedule_df.to_excel(output_file_path, index=False)
new_schedule_df

,id,name,sun-0600am-1030am,sun-1030am-0300pm,sun-0300pm-0730pm,sun-0730pm-1130pm,mon-0600am-1030am,mon-1030am-0300pm,mon-0300pm-0730pm,mon-0730pm-1130pm,...,thu-0300pm-0730pm,thu-0730pm-1130pm,fri-0600am-1030am,fri-1030am-0300pm,fri-0300pm-0730pm,fri-0730pm-1130pm,sat-0600am-1030am,sat-1030am-0300pm,sat-0300pm-0730pm,sat-0730pm-1130pm
0,7064287529,Amanda Washington,True,True,True,True,True,False,False,True,...,False,True,True,False,False,True,True,True,True,True
1,5365860717,Joshua Meyer,True,True,True,True,False,False,True,True,...,False,False,False,False,True,True,True,True,True,True
2,6141344355,Randy Gill,True,True,True,True,True,True,False,True,...,False,True,True,True,False,True,True,True,True,True
3,6332792674,Tyler Peterson,True,True,True,True,False,False,True,True,...,False,True,False,False,True,True,True,True,True,True
4,1790210547,Austin Glover,True,True,True,True,False,False,True,False,...,False,False,False,False,True,False,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6319541522,Stephanie Gross,True,True,True,True,False,False,True,True,...,False,True,False,False,True,True,True,True,True,True
96,8793483679,Brittany Jackson,True,True,True,True,True,False,True,False,...,True,True,True,False,True,False,True,True,True,True
97,6779024178,James Evans,True,True,True,True,True,True,False,True,...,True,True,True,True,False,True,True,True,True,True
98,9869890214,Michael Thomas,True,True,True,True,False,True,True,True,...,True,True,False,True,True,True,True,True,True,True


# Proposed Solution

We will test out two types of algorithms for this problem. 
1. **Constraint Satisfaction Problem (CSP)**:
* We will take each students’ assigned shifts as a variable, and the possible shift slots as the domain. The basic constraints are that (1) each student can only work 3 or 4 shifts per week, (2) between 6am - 12am, at least 6 student workers will always be present, (3) no shifts can overlap with class times, and (4) a shift always starts on the clock or at half and hour. 
* We first assign shifts to the student with least available time slots, and recursively assign shifts to the next student. We will propagate this updated constraint to the domains of remaining students. If there are any conflicts, we will backtrack to the last assignment. Finally, we check that all students are assigned shifts without conflicts, and verify our solution. 
2. **Reinforcement Learning with Q-learning**:
* We will define the states as the current schedule configuration, and actions as assigning a shift. We will give rewards for satisfying each constraint (e.g., add score for having 3 or 4 shifts per week, minus score for having consecutive shifts). We will test out different learning rates and epsilon scores to find the optimal policy. Finally, we check that all students are assigned shifts without conflicts. 

# Evaluation Metrics

For each schedule result, we will test the following metrics on the training and testing dataset. 
1. **Schedule feasibility:**
* Measures the percentage of constraints satisfied by the algorithm over the total number of constraints which would include students having 3-4 working shifts, no more than one shift per day, avoid consecutive opening and closing shifts, having enough student workers during peak hours, and having 1-2 leads per shift.

* Schedule feasibility = (Number of satisfied constraints/Total constraints) x number of students
Ex: IF there are 350 satisfied constraints out of 400 total constraints, the constraint satisfaction would be 87.5%

2. **Student workload balance:** 
* How even the workload is distributed to students based on their number of shifts and their schedule. For example, the score would be low if certain students are only assigned 2 shifts and others are assigned 5. 

* Workload balance = standard deviation of number of shifts = std (1/n * sum (Si - S_mean)^2), where we have n students in total, i goes from 1 to n, and S represents the number of shifts per student per week. 
3. **Computational efficiency:** 
* Time taken to generate schedules. We want the tool to be generalized to larger facilities, so a faster calculation would have a better performance. 
4. **Lead presence score:** 
* Metric that measures the amount of shifts that have 1-2 leads present
* Lead Presence score = (Number of shifts 1-2 leads/Total number of shifts) x 100
    * Ex: If there are 100 shifts and 85 of them have 1-2 leads present, then the lead presence score would be 85%

# Results

The following illustrates our coding process:

## Setup

In [7]:
import numpy as np, pandas as pd
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

from tqdm import tqdm

In [8]:
# Read in availability schedule from the data
ava_matrix = pd.read_excel("availability_schedule.xlsx")

# Set weight settings
weights = {
    "empty_shift_penalty": 10,
    "availability_penalty": 10,
    "student_count_penalty": 1,
    "underworked_penalty": 2,
    "overworked_penalty": 2
}

## Evaluation
Our evaluation function compares the schedule matrix (work shift schedule) with the availiblity matrix to determine the penalties it should mark. The first two are hard constraints, which penalize 10 points for having an empty shift or a scheduling conflict. The last three are soft constraints, which penalize for having the wrong number of workers during a shift, having students with too few hours, and having students with too many hours.

In [9]:
def evaluate_schedule(sch_matrix, ava_matrix):
    score = 0
    # Hard constraints
    for shift in range(2, 30):
        if sum(sch_matrix.iloc[:, shift]) == 0:
            score -= weights['empty_shift_penalty']
        for student in range(100):
            if sch_matrix.iloc[student, shift] == True and ava_matrix.iloc[student, shift] == False:
                score -= weights['availability_penalty']
                
    # Preferences
    for shift in range(2, 30):
        num_students = sum(sch_matrix.iloc[:, shift])
        score -= weights['student_count_penalty'] * abs(num_students - 6)

    for student in range(100):
        num_shifts = sum(sch_matrix.iloc[student, 2:])
        if num_shifts < 3:
            score -= weights['underworked_penalty'] * (3 - num_shifts)
        elif num_shifts > 4:
            score -= weights['overworked_penalty'] * (num_shifts - 4)
    
    return score

## Benchmarks

To compare our results, we came up with two different schedules before making an improved one. The first is a trivial solution, which only abides by the hard constraints. This is achieved by simply making the schedule matrix equivalent to the availability matrix, or in other words, scheduling every student whenever possible.

In [10]:
# "Benchmark", score for the easiest possible schedule to come up with
evaluate_schedule(ava_matrix, ava_matrix)

-4867

The result is a very low score, which we will consider the absolute bare minimum to beat.

The second benchmark is achieved by a directed scheduling algorith, which chooses a random schedule based on the constraints, but does not utilize AI methods. This function considers all five constraints, and introduces some randomness in order to obtain a variety of schedules:

In [11]:
# Short helper function to randomly pick an item from a list
def choose(lst):
    return np.random.choice(lst)

# Create an empty schedule, to which we will add shifts
empty_sch = ava_matrix.copy()
empty_sch.iloc[:, 2:] = False

# Generate a random schedule that should achieve a decent score
def generate_random_schedule(ava_matrix, initial_schedule=None):
    if initial_schedule is None:
        sch_matrix = empty_sch.copy()
    else:
        sch_matrix = initial_schedule.copy()
        
    for shift in range(2, 30):
        # Shuffle rows of the availability schedule to get randomness
        ava_matrix = ava_matrix.sample(frac=1).reset_index(drop=True)
        
        for ava_id in range(100):
            # Typically, we would stop at 6, but this adds randomness:
            if sum(sch_matrix.iloc[:, shift]) >= choose((5, 6, 7, 8, 9)):
                break
            
            student_id = ava_matrix.iloc[ava_id]["id"]
            sch_id = sch_matrix[sch_matrix["id"] == student_id].index[0] # get location of student in schedule matrix
            
            # Typically, we would stop at 4, but this adds randomness:
            if sum(sch_matrix.iloc[sch_id, 2:]) >= choose((2, 3, 4, 5)):
                continue
            
            # If the student is available then...
            if ava_matrix.iloc[ava_id, shift] == True:
                # If the student is not already working...
                if sch_matrix.iloc[sch_id, shift] == False: 
                    # Assign the student to that shift
                    sch_matrix.iloc[sch_id, shift] = True 
                    
    return sch_matrix

In [12]:
# The average score for random datasets can be thought of 
# as the benchmark for a non-AI algorithm
scores = []
for _ in tqdm(range(100)):
    test_schedule = generate_random_schedule(ava_matrix)
    evaluation = evaluate_schedule(test_schedule, ava_matrix)
    scores.append(evaluation)
    
avg_score = np.mean(scores)
avg_score

100%|█████████████████████████████████████████| 100/100 [00:08<00:00, 11.30it/s]


-289.58

The resulting score is much better than the previous benchmark, and should be difficult to beat.

However, this evaluation function only gives us an overall picture of the performance, and does not tell us which errors are occuring the most. To determine this, we divided the evaluation function into three parts to determine what was primarily contributing to the overall penalty.

In [13]:
# Sub-evaluation-functions that tell you the penalty for each type of error
def ev_count(sch_matrix, ava_matrix):
    score = 0
    for shift in range(2, 30):
        num_students = sum(sch_matrix.iloc[:, shift])
        score -= weights['student_count_penalty'] * abs(num_students - 6)
    return score

def ev_under(sch_matrix, ava_matrix):
    score = 0
    for student in range(100):
        num_shifts = sum(sch_matrix.iloc[student, 2:])
        if num_shifts < 3:
            score -= weights['underworked_penalty']
    return score

def ev_over(sch_matrix, ava_matrix):
    score = 0
    for student in range(100):
        num_shifts = sum(sch_matrix.iloc[student, 2:])
        if num_shifts > 4:
            score -= weights['underworked_penalty']
    return score

In [14]:
# View the average scores for each type of penalty
scores = []
for _ in tqdm(range(100)):
    test_schedule = generate_random_schedule(ava_matrix)
    eval_count = ev_count(test_schedule, ava_matrix)
    eval_under = ev_under(test_schedule, ava_matrix)
    eval_over = ev_over(test_schedule, ava_matrix)
    scores.append((eval_count, eval_under, eval_over))
    
avg_count_penalty, avg_under_penalty, avg_over_penalty = np.array(scores).mean(axis=0)

print("student_count_penalty:", avg_count_penalty)
print("underworked_penalty:", avg_under_penalty)
print("overworked_penalty:", avg_over_penalty)

100%|█████████████████████████████████████████| 100/100 [00:07<00:00, 13.34it/s]

student_count_penalty: -20.86
underworked_penalty: -151.64
overworked_penalty: -1.32


From these results, it seems that the primary issue contributing to the penalty is the underworked penalty, meaning that there is generally a problem of students not having enough (3 to 4) shifts. 

With this in mind, the way we decided to improve this was by adding incrementally shifts to the schedule. We implemented this in a greedy algorithm described below:

In [15]:
# Function to sort matrix: underworked students on top, underworked shifts to the left
def sort_matrix(sch_matrix):
    sch_matrix = sch_matrix.iloc[:, 2:].T.reset_index(drop=True).T
    
    sch_matrix['shift_count'] = sch_matrix.sum(axis=1)
    sch_matrix = sch_matrix.sort_values(by="shift_count")
    sch_matrix = sch_matrix.drop("shift_count", axis=1)
    
    # Transpose and do the same
    sch_matrix = sch_matrix.T
    sch_matrix['student_count'] = sch_matrix.sum(axis=1)
    sch_matrix = sch_matrix.sort_values(by="student_count")
    sch_matrix = sch_matrix.drop("student_count", axis=1)
    
    return sch_matrix.T

def add_shifts(sch_matrix, ava_matrix):
    
    # Sort the matrix to keep account of the rows and columns
    sch_sorted = sort_matrix(sch_matrix)
    students_ordered = sch_sorted.index
    columns_ordered = sch_sorted.columns + 2
    
    # Greedy algorithm
    for student_id in tqdm(students_ordered):
        for shift_id in columns_ordered:
            # If the student is not already working then...
            if sch_matrix.iloc[student_id, shift_id] == False:
                # If the student is available then...
                if ava_matrix.iloc[student_id, shift_id] == True:
                    # Test out a new schedule with that slot filled
                    new_sch_matrix = sch_matrix.copy()
                    new_sch_matrix.iloc[student_id, shift_id] = True
                    
                    # Evaluate the new schedule
                    current_score = evaluate_schedule(sch_matrix, ava_matrix)
                    new_score = evaluate_schedule(new_sch_matrix, ava_matrix)
                    
                    # If the new schedule is better, update the original schedule matrix
                    if new_score > current_score:
                        sch_matrix = new_sch_matrix
    return sch_matrix

In [16]:
improved_sch = add_shifts(test_schedule, ava_matrix)

100%|█████████████████████████████████████████| 100/100 [01:17<00:00,  1.29it/s]


To compare this result with the non-AI benchmark:

In [17]:
# compare non-AI benchmark with AI result
new_score = evaluate_schedule(improved_sch, ava_matrix)
print("Non-AI benchmark:", avg_score)
print("New result:", new_score)

Non-AI benchmark: -289.58
New result: -145


The new result is almost twice as good as the benchmark.

# Discussion

### Interpreting the result

#### Main Point: Improved Scheduling Efficiency through the greedy approach
- The result of this project is the improvement in scheduling by using the greedy approach to iteratively add shifts based on the evaluation function. The improved scores showcase that through this method, each shift has enough student workers while taking into account their class schedules and also making sure to not overwork or underwork any students.

#### Minimizing  empty shifts:
- The evaluation function has a penalty for the empty shifts so by adding a constraint that there needs to be at least one student per shift, the schedule score would be greater. There are more empty shifts in the initial random schedules compared to the improved schedules due to the constraint.

#### Following the availability constraints:
- The greedy approach follows the constraints of students' class schedules which means that the scheduler would only give shifts to students that are open during the given time frames. Any time a student's availability is ignored, there is a penalty. There are less violations when it comes to availability constraints with the improved schedules compared to the initial schedules.

#### Overworking or underworking students and shift coverage:
- Each shift needs to have 6 people working at all times to ensure each shift is adequately staffed and the scheduler needs to take into account students' class schedules as well. Looking at the results, there are generally more people who are underworked compared to overworked. The algorithm helps reduce overworking and underworking students throughout the week by having a penalty whenever a student has either less than 3 shifts or more than 4 shifts. 

### Limitations

Some limitations to this project is the simple evalutation scoring system based on predefined penalties that might not fully encapsulate the way a dining hall may operate. There may be instances where there are specific preferences or constraints that won't be captured by the evalutation scoring system. The scalability of this algorithm may be inefficient as the number of students increase. The dataset used was only a sample size of 100. This approach does not use that many hyperparameters which can create usable schedules, but not the most optimal one. Adding more constraints for the model such as adding more students to peak hours and not allowing students to have a closing shift right before an opening shift to use would increase the likelihood for a good schedule. 

### Ethics & Privacy

As mentioned before, we are creating synthetic data to train/test our tool. As such, we are not using real student data and are not concerned about privacy violations. We will also not provide any real information about students, classes, or facilities in our data.

We do not intend to release our tool for actual use, but we will be very clear on any faults it has in our final report. We will also make it clear that it has been trained on synthetic data and not real student data, so while it will hopefully generalize to the real world, that is not what it will have been trained on.

### Conclusion

The project effectively displayed the use of a greedy approach to improve the shift scheduling of students working in dining halls. With many constraints such as reducing empty shifts, balancing shifts amongst the students working there, and their overall availabilty throughout the week, there are notable changes to the scheduler. The higher evaluation scores indicate that the algorithm can adjust to the given constraints as well as successfully elevate the shift assignments.

Shift scheduling can be difficult when it comes to any job. In the context of workforce scheduling (WSP), this greedy approach can be used in customer service or healthcare to help maximize efficiency in scheduling for all the workers while making sure there are enough staff per shift.

This model is fairly sipmle and straight forward so to improve on the current approach, future work could include taking into account student preferences and other fairness constraints that would create a more realistic way of scheduling students while possibly exploring reinforcement learning. Adding constraints for peak hours, shift leads, and times when students have emergencies and can not attend their shift are all ways to improve the performance of the scheduler as well as its scalability.

# Footnotes
<a name="footnote1">[1]:</a> [Workforce Scheduling](https://www.researchgate.net/publication/2333359_Workforce_Scheduling)<br>
<a name="footnote2">[2]:</a> [Heuristic Algorithm for Workforce Scheduling Problems](https://www.researchgate.net/publication/46189793_Heuristic_Algorithm_for_Workforce_Scheduling_Problems)<br>
<a name="footnote3">[3]:</a> [Constraint-Satisfaction Problems in Scheduling](http://eil2.mie.utoronto.ca/wp-content/uploads/scheduling/papers/fox-ecai90.pdf)<br>
<a name="footnote4">[4]:</a> [Reinforcement Learning in Scheduling](https://www.sciencedirect.com/science/article/pii/S2212827122001846)<br>

ChatGPT was used for support with coding and writing.